In [131]:
from copy import copy
from modules.lib import parse_genome_property_file
from modules.genome_property import build_genome_property_connections

genome_property_flat_file_path = '../genome-properties/flatfiles/genomeProperties.txt'
genome_property_assignment_file_path = './LONGFORM_REPORT_'

with open(genome_property_flat_file_path) as genome_property_file:
    global_properties = parse_genome_property_file(genome_property_file)

In [132]:
organism_properties = {}
with open(genome_property_assignment_file_path) as genome_property_assignment_file:
    current_property_id = ''
    current_step = ''
    step_result = False
    property_result = False
    current_steps = []
    
    for line in genome_property_assignment_file:
        if 'PROPERTY:' in line:
            current_property_id = line.split(':')[1].strip()
        elif 'STEP NUMBER:' in line:
            current_step_number = int(line.split(':')[1].strip())
        elif 'RESULT:' in line:
            result = line.split(':')[1].strip().lower()

            if 'yes' in result:
                result = True
            else:
                result = False

            if 'STEP' in line:
                if result:
                    current_steps.append(current_step_number)
            else:
                if result:
                    organism_properties[current_property_id] = current_steps
                    current_steps = []
        else:
            continue

In [133]:
len(organism_properties)

216

In [134]:
organism_property_ids = set(organism_properties.keys())

GenProp0065, Type: CATEGORY, Name: Genome properties, Thresh: 0, References: False, Databases: False, Steps: True, Parents: False, Children: True, Public: False

In [142]:
def get_organism_genome_properties_sub_tree(organism_genome_property_ids, global_genome_properties_tree):
    organism_properties_tree = {}

    for identifier in organism_genome_property_ids:
        genome_property = copy(global_genome_properties_tree.get(identifier))
        if genome_property:
            genome_property.parents = []
            genome_property.child = []
            organism_properties_tree[identifier] = genome_property

    category_genome_property_ids = []
    for genome_property in global_genome_properties_tree.values():
        if genome_property.type == 'CATEGORY':
            category_genome_property_ids.append(genome_property.id)
            genome_property_copy = copy(genome_property)
            genome_property_copy.parents = []
            genome_property_copy.children = []
            organism_properties_tree[genome_property_copy.id] = genome_property_copy

    build_genome_property_connections(organism_properties_tree)

    nodes_to_delete = True
    while nodes_to_delete:
        category_property_ids_to_delete = []
        for identifier in category_genome_property_ids:
            category_genome_property = organism_properties_tree[identifier]
            if not category_genome_property.children:
                category_property_ids_to_delete.append(identifier)

        if category_property_ids_to_delete:
            for identifier in category_property_ids_to_delete:
                genome_property = organism_properties_tree[identifier]
                parents = genome_property.parents
                if parents:
                    for parent in parents:
                        parent.children.remove(genome_property)
                category_genome_property_ids.remove(identifier)
                del organism_properties_tree[identifier]
        else:
            nodes_to_delete = False

    return organism_properties_tree

In [145]:
subtree = get_organism_genome_properties_sub_tree(organism_property_ids, global_properties)